# MLP Classification with MPQA Dataset
<hr>

We will build a text classification model using Multi Layer Perceptron on the MPQA Dataset. Since there is no standard train/test split for this dataset, we will use 10-Fold Cross Validation (CV). 

## Load the library

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold
from tensorflow.keras.preprocessing.text import Tokenizer

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False
# nltk.download('twitter_samples')


Bad key text.latex.unicode in file C:\Users\Diardano Raihan\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.frameon in file C:\Users\Diardano Raihan\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 423 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key pgf.debug in file C:\Users\Diardano Raihan\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 444 ('pgf.debug           : False')
You probably need to get an updated matplotlibrc file from
h

In [2]:
tf.config.list_physical_devices('GPU') 

[]

## Load the Dataset

In [3]:
corpus = pd.read_pickle('../../../0_data/MPQA/MPQA.pkl')
corpus.label = corpus.label.astype(int)
print(corpus.shape)
corpus

(10606, 3)


,sentence,label,split
0,complaining,0,train
1,failing to support,0,train
2,desperately needs,0,train
3,many years of decay,0,train
4,no quick fix,0,train
...,...,...,...
10601,urged,1,train
10602,strictly abide,1,train
10603,hope,1,train
10604,strictly abide,1,train


In [4]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10606 entries, 0 to 10605
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  10606 non-null  object
 1   label     10606 non-null  int32 
 2   split     10606 non-null  object
dtypes: int32(1), object(2)
memory usage: 207.3+ KB


In [5]:
corpus.groupby( by='label').count()

,sentence,split
label,,
0,7294,7294
1,3312,3312


In [6]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

In [7]:
sentences[0]

'complaining'

<!--## Split Dataset-->

# Data Preprocessing: Word2Vec Static
<hr>

Preparing data for word embedding, especially for pre-trained word embedding like Word2Vec or GloVe, __don't use standard preprocessing steps like stemming or stopword removal__. Compared to our approach on cleaning the text when doing word count based feature extraction (e.g. TFIDF) such as removing stopwords, stemming etc, now we will keep these words as we do not want to lose such information that might help the model learn better.

__Tomas Mikolov__, one of the developers of Word2Vec, in _word2vec-toolkit: google groups thread., 2015_, suggests only very minimal text cleaning is required when learning a word embedding model. Sometimes, it's good to disconnect
In short, what we will do is:
- Puntuations removal
- Lower the letter case
- Tokenization

The process above will be handled by __Tokenizer__ class in TensorFlow

## Load Pre-trained Word Embedding: Word2Vec

__1. Load `Word2Vec` Pre-trained Word Embedding__

__Using and updating pre-trained embeddings__
* In this part, we will create an Embedding layer in Tensorflow Keras using a pre-trained word embedding called Word2Vec 300-d tht has been trained 100 bilion words from Google News.
* In this part,  we will leave the embeddings fixed instead of updating them (dynamic).

In [8]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)

In [9]:
# Access the dense vector value for the word 'handsome'
# word2vec.word_vec('handsome') # 0.11376953
word2vec.word_vec('cool') # 1.64062500e-01

array([ 1.64062500e-01,  1.87500000e-01, -4.10156250e-02,  1.25000000e-01,
       -3.22265625e-02,  8.69140625e-02,  1.19140625e-01, -1.26953125e-01,
        1.77001953e-02,  8.83789062e-02,  2.12402344e-02, -2.00195312e-01,
        4.83398438e-02, -1.01074219e-01, -1.89453125e-01,  2.30712891e-02,
        1.17675781e-01,  7.51953125e-02, -8.39843750e-02, -1.33666992e-02,
        1.53320312e-01,  4.08203125e-01,  3.80859375e-02,  3.36914062e-02,
       -4.02832031e-02, -6.88476562e-02,  9.03320312e-02,  2.12890625e-01,
        1.72119141e-02, -6.44531250e-02, -1.29882812e-01,  1.40625000e-01,
        2.38281250e-01,  1.37695312e-01, -1.76757812e-01, -2.71484375e-01,
       -1.36718750e-01, -1.69921875e-01, -9.15527344e-03,  3.47656250e-01,
        2.22656250e-01, -3.06640625e-01,  1.98242188e-01,  1.33789062e-01,
       -4.34570312e-02, -5.12695312e-02, -3.46679688e-02, -8.49609375e-02,
        1.01562500e-01,  1.42578125e-01, -7.95898438e-02,  1.78710938e-01,
        2.30468750e-01,  

In [10]:
def training_words_in_word2vector(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    vocab_size = len(word_to_index) + 1
    count = 0
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            count+=1
            
    return print('Found {} words present from {} training vocabulary in the set of pre-trained word vector'.format(count, vocab_size))

In [11]:
oov_tok = '<OOV>'

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
training_words_in_word2vector(word2vec, word_index)

Found 6083 words present from 6236 training vocabulary in the set of pre-trained word vector


## Define `clean_doc` function
__2. Define a function to clean the document called __`clean_doc()`____

In [12]:
def clean_doc(sentences, word_index):
    clean_sentences = []
    for sentence in sentences:
        sentence = sentence.lower().split()
        clean_word = []
        for word in sentence:
            if word in word_index:
                clean_word.append(word)
        clean_sentence = ' '.join(clean_word)
        clean_sentences.append(clean_sentence)
    return clean_sentences

In [13]:
clean_sentences = clean_doc(sentences, word_index)
clean_sentences[0:3]

['complaining', 'failing to support', 'desperately needs']

## Define `sentence_to_avg` function
__3. Define a `sentence_to_avg` function__

We will use this function to calculate the mean of word embedding representation.

In [14]:
def sentence_to_avg(sentence, word_to_vec_map):
    """
    Converts a sentence (string) into a list of words (strings). Extracts the GloVe representation of each word
    and averages its value into a single vector encoding the meaning of the sentence.
    
    Arguments:
    sentence -- string, one training example from X
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    
    Returns:
    avg -- average vector encoding information about the sentence, numpy-array of shape (50,)
    """
    
    # Step 1: Split sentence into list of lower case words (≈ 1 line)
    words = (sentence.lower()).split()

    # Initialize the average word vector, should have the same shape as your word vectors.
    avg = np.zeros(word2vec.word_vec('i').shape)
    
    # Step 2: average the word vectors. You can loop over the words in the list "words".
    total = 0
    count = 0
    for w in words:
        if w in word_to_vec_map:
            total += word_to_vec_map.word_vec(w)
            count += 1
            
    if count!= 0:
        avg = total/count
    else:
        avg
    return avg

In [15]:
i = word2vec.word_vec('i')[0]
print(word2vec.word_vec('i')[0])
j = word2vec.word_vec('am')[0]
print(word2vec.word_vec('am')[0])
k = word2vec.word_vec('handsome')[0]
print(word2vec.word_vec('handsome')[0])
mean = (i+j+k)/3
print('the mean of word embedding is: ', mean)

-0.22558594
-0.16699219
0.11376953
the mean of word embedding is:  -0.09293619791666667


In [16]:
# Example of the functions used in a sentence
mysentence = 'I am handsome'
sentence_to_avg(mysentence, word2vec)

array([-0.0929362 ,  0.03125   , -0.03914388,  0.09879557,  0.07088598,
        0.03092448, -0.00651042, -0.04437256,  0.08068848,  0.07242838,
        0.00160726, -0.10530599, -0.07389323, -0.08854166,  0.00565592,
        0.15136719, -0.0460612 ,  0.19482422,  0.1101888 ,  0.05924479,
       -0.18457031,  0.00716146,  0.16153972,  0.02437337, -0.01578776,
        0.06119792, -0.25048828,  0.02799479,  0.0853475 , -0.14029948,
        0.13688152, -0.01350911, -0.05493164, -0.01090495,  0.03352864,
        0.09635416, -0.04239909,  0.00777181, -0.1438802 ,  0.06510416,
        0.14560954, -0.11295573,  0.25520834,  0.08833822,  0.14339192,
        0.037028  , -0.02832031, -0.00139872,  0.00309245, -0.17871094,
        0.06852213,  0.07910156,  0.09513346,  0.11425781, -0.00488281,
        0.11051432, -0.01139323, -0.08479818, -0.09277344, -0.03263346,
       -0.00374349,  0.07977295, -0.26416016, -0.05135091,  0.06111654,
       -0.06933594, -0.06486002,  0.18766277, -0.04826609,  0.03

## Encode Sentence into Word2Vec Representation

In [17]:
def encoded_sentences(sentences):

    encoded_sentences = []

    for sentence in sentences:

        encoded_sentence = sentence_to_avg(sentence, word2vec)
        encoded_sentences.append(encoded_sentence)

    encoded_sentences = np.array(encoded_sentences)
    return encoded_sentences

In [18]:
embedded_sentences = encoded_sentences(clean_sentences)
print(embedded_sentences.shape)
embedded_sentences

(10606, 300)


array([[-0.18066406,  0.13476562, -0.13085938, ...,  0.05419922,
         0.17871094,  0.14355469],
       [ 0.06982422,  0.00537109,  0.10314941, ...,  0.1496582 ,
        -0.00378418,  0.01538086],
       [ 0.03710938,  0.13769531, -0.01550293, ...,  0.06748962,
         0.06011963, -0.05154419],
       ...,
       [ 0.01611328,  0.14550781,  0.22265625, ..., -0.08984375,
         0.10986328, -0.0534668 ],
       [-0.21154785, -0.04882812,  0.23828125, ..., -0.02050781,
        -0.07974243, -0.16088867],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

# MLP Model: 1

## Model Definition

In [19]:
def define_model(input_length=300):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense( units=50, activation='relu', input_shape=(input_length,)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense( units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [20]:
model_0 = define_model(300)
model_0.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                15050     
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 15,101
Trainable params: 15,101
Non-trainable params: 0
_________________________________________________________________


## Train and Test the Model

In [21]:
'''
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') >= 0.9):
            print("\nReached 90% accuracy so cancelling training!")
            self.model.stop_training=True
'''
callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [22]:
# Parameter Initialization
oov_tok = "<UNK>"
columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record = pd.DataFrame(columns = columns)

sentences, labels = list(corpus.sentence), list(corpus.label)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the data into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)
    
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Obtain the word to index
    word_index = tokenizer.word_index
    
    # Clean the sentences
    Xtrain = clean_doc(train_x, word_index)
    Xtest = clean_doc(test_x, word_index)

    # Encode the sentences into word embedding average representation
    Xtrain = encoded_sentences(Xtrain)
    Xtest = encoded_sentences(Xtest)
    
    # Define the input shape
    model = define_model(Xtrain.shape[1])

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=40, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))
    print()

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record = record.append(temp, ignore_index=True)
print()
print(record)
print()

Training 1: 


C:\Users\Diardano Raihan\Anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass shuffle=True as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


Epoch 1/40
299/299 [==============================] - 1s 2ms/step - loss: 0.4113 - accuracy: 0.8195 - val_loss: 0.3367 - val_accuracy: 0.8633
Epoch 2/40
299/299 [==============================] - 0s 1ms/step - loss: 0.3143 - accuracy: 0.8760 - val_loss: 0.3265 - val_accuracy: 0.8662
Epoch 3/40
299/299 [==============================] - 0s 931us/step - loss: 0.2999 - accuracy: 0.8831 - val_loss: 0.3224 - val_accuracy: 0.8652
Epoch 4/40
299/299 [==============================] - 0s 1ms/step - loss: 0.2896 - accuracy: 0.8894 - val_loss: 0.3198 - val_accuracy: 0.8690
Epoch 5/40
299/299 [==============================] - 0s 996us/step - loss: 0.2827 - accuracy: 0.8915 - val_loss: 0.3185 - val_accuracy: 0.8680
Epoch 6/40
299/299 [==============================] - 0s 1ms/step - loss: 0.2759 - accuracy: 0.8939 - val_loss: 0.3196 - val_accuracy: 0.8671
Epoch 7/40
299/299 [==============================] - 0s 906us/step - loss: 0.2724 - accuracy: 0.8939 - val_loss: 0.3139 - val_accuracy: 0.8718


299/299 [==============================] - 1s 4ms/step - loss: 0.4153 - accuracy: 0.8155 - val_loss: 0.3443 - val_accuracy: 0.8765
Epoch 2/40
299/299 [==============================] - 1s 3ms/step - loss: 0.3164 - accuracy: 0.8718 - val_loss: 0.3334 - val_accuracy: 0.8746
Epoch 3/40
299/299 [==============================] - 1s 2ms/step - loss: 0.3009 - accuracy: 0.8837 - val_loss: 0.3229 - val_accuracy: 0.8869
Epoch 4/40
299/299 [==============================] - 1s 2ms/step - loss: 0.2886 - accuracy: 0.8867 - val_loss: 0.3202 - val_accuracy: 0.8860
Epoch 5/40
299/299 [==============================] - 1s 2ms/step - loss: 0.2832 - accuracy: 0.8893 - val_loss: 0.3193 - val_accuracy: 0.8860
Epoch 6/40
299/299 [==============================] - 1s 2ms/step - loss: 0.2773 - accuracy: 0.8924 - val_loss: 0.3150 - val_accuracy: 0.8850
Epoch 7/40
299/299 [==============================] - 1s 2ms/step - loss: 0.2711 - accuracy: 0.8941 - val_loss: 0.3157 - val_accuracy: 0.8860
Epoch 8/40
299/29

299/299 [==============================] - 1s 3ms/step - loss: 0.2516 - accuracy: 0.9021 - val_loss: 0.3278 - val_accuracy: 0.8775
Epoch 13/40
299/299 [==============================] - 1s 3ms/step - loss: 0.2462 - accuracy: 0.9059 - val_loss: 0.3313 - val_accuracy: 0.8812
Epoch 14/40
299/299 [==============================] - 1s 3ms/step - loss: 0.2468 - accuracy: 0.9058 - val_loss: 0.3296 - val_accuracy: 0.8775
Epoch 15/40
299/299 [==============================] - 1s 3ms/step - loss: 0.2443 - accuracy: 0.9083 - val_loss: 0.3288 - val_accuracy: 0.8803
Epoch 16/40
299/299 [==============================] - 1s 3ms/step - loss: 0.2384 - accuracy: 0.9080 - val_loss: 0.3296 - val_accuracy: 0.8831
Epoch 17/40
299/299 [==============================] - 1s 3ms/step - loss: 0.2357 - accuracy: 0.9093 - val_loss: 0.3286 - val_accuracy: 0.8822
Epoch 18/40
299/299 [==============================] - 1s 3ms/step - loss: 0.2334 - accuracy: 0.9119 - val_loss: 0.3256 - val_accuracy: 0.8850
Epoch 19/40

299/299 [==============================] - 1s 3ms/step - loss: 0.2184 - accuracy: 0.9189 - val_loss: 0.3285 - val_accuracy: 0.8745
Epoch 26/40
299/299 [==============================] - 1s 3ms/step - loss: 0.2156 - accuracy: 0.9161 - val_loss: 0.3298 - val_accuracy: 0.8717
Epoch 27/40
299/299 [==============================] - 1s 3ms/step - loss: 0.2133 - accuracy: 0.9172 - val_loss: 0.3315 - val_accuracy: 0.8708
Epoch 28/40
299/299 [==============================] - 1s 3ms/step - loss: 0.2138 - accuracy: 0.9190 - val_loss: 0.3344 - val_accuracy: 0.8736
Epoch 29/40
299/299 [==============================] - 1s 3ms/step - loss: 0.2056 - accuracy: 0.9221 - val_loss: 0.3387 - val_accuracy: 0.8726
Epoch 30/40
299/299 [==============================] - 1s 3ms/step - loss: 0.2111 - accuracy: 0.9208 - val_loss: 0.3354 - val_accuracy: 0.8717
Epoch 31/40
299/299 [==============================] - 1s 3ms/step - loss: 0.2052 - accuracy: 0.9228 - val_loss: 0.3355 - val_accuracy: 0.8708
Epoch 00031

299/299 [==============================] - 1s 2ms/step - loss: 0.2524 - accuracy: 0.9009 - val_loss: 0.3472 - val_accuracy: 0.8764
Epoch 13/40
299/299 [==============================] - 1s 2ms/step - loss: 0.2494 - accuracy: 0.9038 - val_loss: 0.3435 - val_accuracy: 0.8736
Epoch 14/40
299/299 [==============================] - 1s 2ms/step - loss: 0.2467 - accuracy: 0.9070 - val_loss: 0.3453 - val_accuracy: 0.8755
Epoch 15/40
299/299 [==============================] - 1s 2ms/step - loss: 0.2416 - accuracy: 0.9094 - val_loss: 0.3430 - val_accuracy: 0.8708
Epoch 16/40
299/299 [==============================] - 1s 2ms/step - loss: 0.2412 - accuracy: 0.9088 - val_loss: 0.3466 - val_accuracy: 0.8745
Epoch 17/40
299/299 [==============================] - 1s 2ms/step - loss: 0.2392 - accuracy: 0.9073 - val_loss: 0.3498 - val_accuracy: 0.8717
Epoch 18/40
299/299 [==============================] - 1s 2ms/step - loss: 0.2345 - accuracy: 0.9098 - val_loss: 0.3466 - val_accuracy: 0.8726
Epoch 19/40

## Summary

In [23]:
record

,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,87.935907,88.312912,88.689917,86.145145,88.501412,86.71065,87.73585,87.830186,85.849059,87.641507,87.535254


In [24]:
report = record
report = report.to_excel('Emb_MLP_MPQA.xlsx', sheet_name='model_1')

<hr>

# MLP Model: 2

## Model Definition

In [25]:
def define_model_2(input_length=300):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense( units=100, activation='relu', input_shape=(input_length,)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense( units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [26]:
model_1 = define_model_2(300)
model_1.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 100)               30100     
_________________________________________________________________
dropout_11 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 101       
Total params: 30,201
Trainable params: 30,201
Non-trainable params: 0
_________________________________________________________________


## Train and Test the Model

In [27]:
callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=1, 
                                             mode='auto', restore_best_weights=True)

In [28]:
# Parameter Initialization
oov_tok = "<UNK>"
columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record2 = pd.DataFrame(columns = columns)

sentences, labels = list(corpus.sentence), list(corpus.label)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the data into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)
    
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Obtain the word to index
    word_index = tokenizer.word_index
    
    # Clean the sentences
    Xtrain = clean_doc(train_x, word_index)
    Xtest = clean_doc(test_x, word_index)

    # Encode the sentences into word embedding average representation
    Xtrain = encoded_sentences(Xtrain)
    Xtest = encoded_sentences(Xtest)
    
    # Define the input shape
    model = define_model_2(Xtrain.shape[1])

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=40, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))
    print()

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record2 = record2.append(temp, ignore_index=True)
print()
print(record2)
print()

Training 1: 


C:\Users\Diardano Raihan\Anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass shuffle=True as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


Epoch 1/40
299/299 [==============================] - 1s 3ms/step - loss: 0.3894 - accuracy: 0.8276 - val_loss: 0.3543 - val_accuracy: 0.8596
Epoch 2/40
299/299 [==============================] - 1s 2ms/step - loss: 0.3024 - accuracy: 0.8800 - val_loss: 0.3517 - val_accuracy: 0.8577
Epoch 3/40
299/299 [==============================] - 1s 2ms/step - loss: 0.2861 - accuracy: 0.8902 - val_loss: 0.3416 - val_accuracy: 0.8662
Epoch 4/40
299/299 [==============================] - 1s 2ms/step - loss: 0.2778 - accuracy: 0.8939 - val_loss: 0.3426 - val_accuracy: 0.8615
Epoch 5/40
299/299 [==============================] - 1s 2ms/step - loss: 0.2664 - accuracy: 0.8966 - val_loss: 0.3396 - val_accuracy: 0.8615
Epoch 6/40
299/299 [==============================] - 1s 2ms/step - loss: 0.2602 - accuracy: 0.8983 - val_loss: 0.3477 - val_accuracy: 0.8615
Epoch 7/40
299/299 [==============================] - 1s 2ms/step - loss: 0.2544 - accuracy: 0.9029 - val_loss: 0.3445 - val_accuracy: 0.8596
Epoch 

299/299 [==============================] - 1s 2ms/step - loss: 0.2063 - accuracy: 0.9233 - val_loss: 0.3236 - val_accuracy: 0.8746
Epoch 20/40
299/299 [==============================] - 1s 2ms/step - loss: 0.1990 - accuracy: 0.9251 - val_loss: 0.3235 - val_accuracy: 0.8737
Epoch 21/40
299/299 [==============================] - 1s 2ms/step - loss: 0.2009 - accuracy: 0.9239 - val_loss: 0.3351 - val_accuracy: 0.8728
Epoch 22/40
299/299 [==============================] - 1s 3ms/step - loss: 0.1973 - accuracy: 0.9257 - val_loss: 0.3240 - val_accuracy: 0.8737
Epoch 23/40
299/299 [==============================] - 1s 3ms/step - loss: 0.1939 - accuracy: 0.9284 - val_loss: 0.3313 - val_accuracy: 0.8690
Epoch 24/40
299/299 [==============================] - 1s 2ms/step - loss: 0.1930 - accuracy: 0.9278 - val_loss: 0.3296 - val_accuracy: 0.8756
Epoch 25/40
299/299 [==============================] - 1s 3ms/step - loss: 0.1858 - accuracy: 0.9315 - val_loss: 0.3278 - val_accuracy: 0.8812
Epoch 26/40

299/299 [==============================] - 1s 3ms/step - loss: 0.2129 - accuracy: 0.9195 - val_loss: 0.3580 - val_accuracy: 0.8596
Epoch 18/40
299/299 [==============================] - 1s 3ms/step - loss: 0.2094 - accuracy: 0.9199 - val_loss: 0.3558 - val_accuracy: 0.8605
Epoch 00018: early stopping
Test Accuracy: 86.52215003967285

Training 6: 
Epoch 1/40
299/299 [==============================] - 2s 7ms/step - loss: 0.3850 - accuracy: 0.8334 - val_loss: 0.3388 - val_accuracy: 0.8615
Epoch 2/40
299/299 [==============================] - 1s 3ms/step - loss: 0.3080 - accuracy: 0.8786 - val_loss: 0.3217 - val_accuracy: 0.8728
Epoch 3/40
299/299 [==============================] - 1s 3ms/step - loss: 0.2907 - accuracy: 0.8873 - val_loss: 0.3184 - val_accuracy: 0.8680
Epoch 4/40
299/299 [==============================] - 1s 3ms/step - loss: 0.2827 - accuracy: 0.8914 - val_loss: 0.3147 - val_accuracy: 0.8784
Epoch 5/40
299/299 [==============================] - 1s 3ms/step - loss: 0.2728 - 

299/299 [==============================] - 2s 5ms/step - loss: 0.2695 - accuracy: 0.8941 - val_loss: 0.3417 - val_accuracy: 0.8717
Epoch 6/40
299/299 [==============================] - 1s 5ms/step - loss: 0.2644 - accuracy: 0.8971 - val_loss: 0.3389 - val_accuracy: 0.8679
Epoch 7/40
299/299 [==============================] - 1s 4ms/step - loss: 0.2543 - accuracy: 0.9023 - val_loss: 0.3388 - val_accuracy: 0.8698
Epoch 8/40
299/299 [==============================] - 1s 4ms/step - loss: 0.2506 - accuracy: 0.9049 - val_loss: 0.3370 - val_accuracy: 0.8670
Epoch 9/40
299/299 [==============================] - 1s 4ms/step - loss: 0.2431 - accuracy: 0.9073 - val_loss: 0.3471 - val_accuracy: 0.8660
Epoch 10/40
299/299 [==============================] - 1s 4ms/step - loss: 0.2391 - accuracy: 0.9085 - val_loss: 0.3424 - val_accuracy: 0.8698
Epoch 11/40
299/299 [==============================] - 1s 3ms/step - loss: 0.2373 - accuracy: 0.9099 - val_loss: 0.3404 - val_accuracy: 0.8698
Epoch 12/40
299

299/299 [==============================] - 1s 3ms/step - loss: 0.2417 - accuracy: 0.9063 - val_loss: 0.3315 - val_accuracy: 0.8632
Epoch 11/40
299/299 [==============================] - 1s 3ms/step - loss: 0.2391 - accuracy: 0.9097 - val_loss: 0.3296 - val_accuracy: 0.8651
Epoch 00011: early stopping
Test Accuracy: 86.98112964630127


        acc1       acc2       acc3       acc4      acc5       acc6       acc7  \
0  86.616397  87.747407  88.218659  88.218659  86.52215  88.501412  87.641507   

       acc8      acc9     acc10       AVG  
0  87.73585  86.98113  86.98113  87.51643  



## Summary

In [29]:
record2

,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,86.616397,87.747407,88.218659,88.218659,86.52215,88.501412,87.641507,87.73585,86.98113,86.98113,87.51643


In [30]:
report = record2
report = report.to_excel('Emb_MLP_MPQA_2.xlsx', sheet_name='model_2')

<hr>

# MLP Model: 3

## Model Definition

In [31]:
def define_model_3(input_length=300):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense( units=100, activation='relu', input_shape=(input_length,)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense( units=50, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense( units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [32]:
model_2 = define_model_3(300)
model_2.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_44 (Dense)             (None, 100)               30100     
_________________________________________________________________
dropout_22 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_45 (Dense)             (None, 50)                5050      
_________________________________________________________________
dropout_23 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_46 (Dense)             (None, 1)                 51        
Total params: 35,201
Trainable params: 35,201
Non-trainable params: 0
_________________________________________________________________


## Train and Test the Model

In [33]:
callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=1, 
                                             mode='auto', restore_best_weights=True)

In [34]:
# Parameter Initialization
oov_tok = "<UNK>"
columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record3 = pd.DataFrame(columns = columns)

sentences, labels = list(corpus.sentence), list(corpus.label)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the data into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)
    
    # Define the word_index
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)
    word_index = tokenizer.word_index
    
    # Clean the sentences
    Xtrain = clean_doc(train_x, word_index)
    Xtest = clean_doc(test_x, word_index)

    # Encode the sentences into word embedding average representation
    Xtrain = encoded_sentences(Xtrain)
    Xtest = encoded_sentences(Xtest)
    
    # Define the input shape
    model = define_model_3(Xtrain.shape[1])

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=40, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))
    print()

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record3 = record3.append(temp, ignore_index=True)
print()
print(record3)
print()

Training 1: 


C:\Users\Diardano Raihan\Anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass shuffle=True as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


Epoch 1/40
299/299 [==============================] - 2s 7ms/step - loss: 0.4129 - accuracy: 0.8118 - val_loss: 0.3339 - val_accuracy: 0.8690
Epoch 2/40
299/299 [==============================] - 1s 5ms/step - loss: 0.3119 - accuracy: 0.8787 - val_loss: 0.3365 - val_accuracy: 0.8652
Epoch 3/40
299/299 [==============================] - 1s 4ms/step - loss: 0.2969 - accuracy: 0.8866 - val_loss: 0.3269 - val_accuracy: 0.8699
Epoch 4/40
299/299 [==============================] - 1s 3ms/step - loss: 0.2814 - accuracy: 0.8937 - val_loss: 0.3227 - val_accuracy: 0.8709
Epoch 5/40
299/299 [==============================] - 1s 3ms/step - loss: 0.2744 - accuracy: 0.8954 - val_loss: 0.3233 - val_accuracy: 0.8662
Epoch 6/40
299/299 [==============================] - 1s 3ms/step - loss: 0.2629 - accuracy: 0.8993 - val_loss: 0.3240 - val_accuracy: 0.8709
Epoch 7/40
299/299 [==============================] - 1s 4ms/step - loss: 0.2546 - accuracy: 0.9043 - val_loss: 0.3204 - val_accuracy: 0.8699
Epoch 

299/299 [==============================] - 1s 2ms/step - loss: 0.1574 - accuracy: 0.9420 - val_loss: 0.4287 - val_accuracy: 0.8652
Epoch 23/40
299/299 [==============================] - 1s 3ms/step - loss: 0.1542 - accuracy: 0.9391 - val_loss: 0.4254 - val_accuracy: 0.8680
Epoch 24/40
299/299 [==============================] - 1s 3ms/step - loss: 0.1556 - accuracy: 0.9412 - val_loss: 0.4430 - val_accuracy: 0.8718
Epoch 25/40
299/299 [==============================] - 1s 3ms/step - loss: 0.1476 - accuracy: 0.9445 - val_loss: 0.4383 - val_accuracy: 0.8718
Epoch 26/40
299/299 [==============================] - 1s 3ms/step - loss: 0.1430 - accuracy: 0.9460 - val_loss: 0.4558 - val_accuracy: 0.8690
Epoch 27/40
299/299 [==============================] - 1s 3ms/step - loss: 0.1413 - accuracy: 0.9467 - val_loss: 0.4527 - val_accuracy: 0.8671
Epoch 28/40
299/299 [==============================] - 1s 3ms/step - loss: 0.1378 - accuracy: 0.9452 - val_loss: 0.4672 - val_accuracy: 0.8699
Epoch 29/40

299/299 [==============================] - 1s 3ms/step - loss: 0.1920 - accuracy: 0.9261 - val_loss: 0.4109 - val_accuracy: 0.8511
Epoch 15/40
299/299 [==============================] - 1s 3ms/step - loss: 0.1908 - accuracy: 0.9296 - val_loss: 0.4078 - val_accuracy: 0.8454
Epoch 16/40
299/299 [==============================] - 1s 3ms/step - loss: 0.1748 - accuracy: 0.9361 - val_loss: 0.4094 - val_accuracy: 0.8530
Epoch 17/40
299/299 [==============================] - 1s 3ms/step - loss: 0.1760 - accuracy: 0.9353 - val_loss: 0.4017 - val_accuracy: 0.8567
Epoch 18/40
299/299 [==============================] - 1s 3ms/step - loss: 0.1668 - accuracy: 0.9366 - val_loss: 0.4094 - val_accuracy: 0.8492
Epoch 19/40
299/299 [==============================] - 1s 3ms/step - loss: 0.1659 - accuracy: 0.9382 - val_loss: 0.4218 - val_accuracy: 0.8539
Epoch 20/40
299/299 [==============================] - 1s 3ms/step - loss: 0.1578 - accuracy: 0.9405 - val_loss: 0.4486 - val_accuracy: 0.8492
Epoch 21/40

299/299 [==============================] - 1s 3ms/step - loss: 0.2480 - accuracy: 0.9063 - val_loss: 0.3362 - val_accuracy: 0.8811
Epoch 8/40
299/299 [==============================] - 1s 3ms/step - loss: 0.2390 - accuracy: 0.9105 - val_loss: 0.3280 - val_accuracy: 0.8868
Epoch 9/40
299/299 [==============================] - 1s 3ms/step - loss: 0.2313 - accuracy: 0.9127 - val_loss: 0.3368 - val_accuracy: 0.8792
Epoch 10/40
299/299 [==============================] - 1s 3ms/step - loss: 0.2249 - accuracy: 0.9147 - val_loss: 0.3389 - val_accuracy: 0.8783
Epoch 11/40
299/299 [==============================] - 1s 2ms/step - loss: 0.2169 - accuracy: 0.9177 - val_loss: 0.3418 - val_accuracy: 0.8821
Epoch 12/40
299/299 [==============================] - 1s 2ms/step - loss: 0.2055 - accuracy: 0.9216 - val_loss: 0.3491 - val_accuracy: 0.8811
Epoch 13/40
299/299 [==============================] - 1s 2ms/step - loss: 0.1981 - accuracy: 0.9246 - val_loss: 0.3558 - val_accuracy: 0.8774
Epoch 14/40
2

## Summary

In [35]:
record3

,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,87.087655,87.558907,87.464654,87.65316,88.878417,85.673892,88.867927,86.792451,88.679248,87.73585,87.639216


In [36]:
report = record3
report = report.to_excel('Emb_MLP_MPQA_3.xlsx', sheet_name='model_3')